# Capstone Project:  Restaurant density in Toronto and Lexington, KY

## Introduction

## Data

### Section 1:  Get data about Toronto

Part 1:  Create a dataframe from the Wikipedia page


In [69]:
#Open libraries
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Library import complete')

Library import complete


In [70]:
#Pull the table from Wikipedia
URL = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"

CanCodes = pd.read_html(URL)[0]
CanCodes

#delete rows where Borough='Not assigned'
CanCodes.drop(CanCodes.loc[CanCodes['Borough']=='Not assigned'].index,inplace=True)

CanCodes
CombCanCodes = CanCodes.groupby(["Postcode","Borough"], sort=False).agg( ','.join)
CombCanCodes
print("Shape of dataframe: ",CombCanCodes.shape)

Shape of dataframe:  (103, 1)


Part 2:  Add latitude and longitude to dataframe

In [71]:
#import csv file
loc=pd.read_csv('C:/Users/kevmo/Downloads/Geospatial_Coordinates.csv').set_index("Postal Code")
#loc=pd.read_csv('C:/Users/kkennedy/Downloads/Geospatial_Coordinates.csv').set_index("Postal Code")
#rename column so it matches CombCanCodes
#loc.rename(columns={"Postal Code": "Postcode"},inplace=True)
loc.rename_axis("Postcode", axis='index', inplace=True)
loc


,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [72]:
#Join the dataframes
CanLoc=CombCanCodes.join(loc)

#Borough is an index, so we need to make it a regular column
CanLoc.reset_index('Borough',inplace=True)
CanLoc
for col in CanLoc.columns: 
    print(col) 

Borough
Neighbourhood
Latitude
Longitude


In [73]:
#Get the shape of the dataframe
CanLoc.shape

(103, 4)

Part 3:  Exploring Toronto


In [74]:
#find Toronto
address = 'Toronto, Ontario Canada'

geolocator = Nominatim(user_agent="Toronto_Neighborhoods")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))


The geographical coordinates of Toronto are 43.653963, -79.387207.


In [75]:
for col in CanLoc.columns:
    print(col)

Borough
Neighbourhood
Latitude
Longitude


In [76]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(CanLoc['Latitude'], CanLoc['Longitude'], CanLoc['Borough'], CanLoc['Neighbourhood']):
#for lat, lng, neighborhood in zip(CanLoc['Latitude'], CanLoc['Longitude'], CanLoc['Neighbourhood']):    
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [77]:
#Let's work with downtown Toronto
Downtown_data = CanLoc[CanLoc['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [78]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="Toronto_Neighborhoods")
location = geolocator.geocode(address)
Dlat = location.latitude
Dlong = location.longitude
print('The geographical coordinates of Downtown Toronto are {}, {}.'.format(Dlat, Dlong))


The geographical coordinates of Downtown Toronto are 43.6541737, -79.38081164513409.


In [79]:
# create map of downtown Toronto using latitude and longitude values
map_DownToronto = folium.Map(location=[Dlat, Dlong], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Downtown_data['Latitude'], Downtown_data['Longitude'], Downtown_data['Borough'], Downtown_data['Neighbourhood']):
#for lat, lng, neighborhood in zip(CanLoc['Latitude'], CanLoc['Longitude'], CanLoc['Neighbourhood']):    
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DownToronto)  
    
map_DownToronto

In [80]:
#Set up Foursquare
CLIENT_ID = 'LRO1TXA4U31JLAUVFFCBHTC2XP3A3JIB1CCALQQYTNKBIPMK' # your Foursquare ID
CLIENT_SECRET = 'FU2W2TWVBIRQK1BRFRZSBIQMEJQG0NTMSW1ABLT4XUOQR1RA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentials:
CLIENT_ID: LRO1TXA4U31JLAUVFFCBHTC2XP3A3JIB1CCALQQYTNKBIPMK
CLIENT_SECRET:FU2W2TWVBIRQK1BRFRZSBIQMEJQG0NTMSW1ABLT4XUOQR1RA


In [81]:
#Get the neighborhood's name
Downtown_data.loc[0, 'Neighbourhood']

'Harbourfront,Regent Park'

In [82]:
DT_latitude = Downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
DT_longitude = Downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Downtown_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               DT_latitude, 
                                                               DT_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


In [83]:
#Find venues within a 500 mile radius, but limit to 100 venues

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    DT_latitude, 
    DT_longitude, 
    radius, 
    LIMIT)
url # display URL

#get the result
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e780c28b1cac0001b5daed3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [84]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

#Clean the data and transform it into a dataframe
venues = results['response']['groups'][0]['items']
    

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [85]:
#Count the venues
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

47 venues were returned by Foursquare.


In [86]:
#Let's look at other neighborhoods in downtown Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Toronto_venues = getNearbyVenues(names=Downtown_data['Neighbourhood'],
                                   latitudes=Downtown_data['Latitude'],
                                   longitudes=Downtown_data['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city
Church and Wellesley


In [87]:
#How big is the dataframe?
print(Toronto_venues.shape)
Toronto_venues.head()

(1270, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront,Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [88]:
#How many venues were found?
Toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,82,82,82,82,82,82
"Chinatown,Grange Park,Kensington Market",89,89,89,89,89,89
Christie,17,17,17,17,17,17
Church and Wellesley,88,88,88,88,88,88
"Commerce Court,Victoria Hotel",100,100,100,100,100,100


In [89]:
#Limit venues to restaurants
Restonly='Restaurant|Caf|Joint|Gastropub|Diner|Pizza|Breakfast|Sandwich'
Toronto_venues=Toronto_venues[Toronto_venues['Venue Category'].str.contains(Restonly)]
Toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,"Harbourfront,Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,"Harbourfront,Regent Park",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
19,"Harbourfront,Regent Park",43.65426,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
21,"Harbourfront,Regent Park",43.65426,-79.360636,ODIN Cafe + Bar,43.656739,-79.356503,Café
23,"Harbourfront,Regent Park",43.65426,-79.360636,Cocina Economica,43.654959,-79.365657,Mexican Restaurant
...,...,...,...,...,...,...,...
1254,Church and Wellesley,43.66586,-79.383160,The Salad House,43.669508,-79.386061,Mediterranean Restaurant
1255,Church and Wellesley,43.66586,-79.383160,Rolltation,43.669388,-79.386566,Sushi Restaurant
1259,Church and Wellesley,43.66586,-79.383160,Kokoni Izakaya,43.664181,-79.380258,Japanese Restaurant
1260,Church and Wellesley,43.66586,-79.383160,Asahi Sushi,43.669874,-79.382943,Sushi Restaurant


In [90]:
Toronto_venues.shape


(479, 7)

In [91]:
#How many unique categories?
print('There are {} unique categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 56 unique categories.


In [92]:
#Let's look at each neighborhood

# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bed & Breakfast,Belgian Restaurant,Brazilian Restaurant,Breakfast Spot,Burger Joint,...,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,"Harbourfront,Regent Park",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
19,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
#How big is the dataframe?
Toronto_onehot.shape

(479, 57)

In [94]:
Toronto_restaurants=Toronto_onehot.filter(regex=Restonly)
Toronto_restaurants

,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bed & Breakfast,Belgian Restaurant,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,...,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1255,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1259,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [95]:
#Group everything by neighborhood and take the average of each category
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,BBQ Joint,Bed & Breakfast,Belgian Restaurant,Brazilian Restaurant,Breakfast Spot,Burger Joint,...,Restaurant,Sandwich Place,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.048780,0.048780,0.000000,0.00000,0.00000,0.024390,0.048780,0.024390,...,0.121951,0.024390,0.048780,0.048780,0.000000,0.097561,0.000000,0.048780,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.066667,0.00000,0.00000,0.000000,0.066667,0.000000,...,0.133333,0.000000,0.133333,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000
2,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.052632,0.000000,...,0.157895,0.052632,0.000000,0.000000,0.052632,0.052632,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.027778,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.083333,...,0.027778,0.083333,0.027778,0.027778,0.000000,0.055556,0.000000,0.027778,0.000000,0.000000
4,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.02439,0.02439,0.000000,0.024390,0.048780,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,0.000000,0.097561,0.121951,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.027027,0.027027,0.000000,0.000000,0.00000,0.00000,0.000000,0.027027,0.027027,...,0.081081,0.000000,0.027027,0.081081,0.000000,0.027027,0.027027,0.000000,0.027027,0.027027
7,"Commerce Court,Victoria Hotel",0.000000,0.088889,0.022222,0.000000,0.00000,0.00000,0.000000,0.022222,0.022222,...,0.155556,0.022222,0.066667,0.000000,0.000000,0.044444,0.000000,0.044444,0.000000,0.000000
8,"Design Exchange,Toronto Dominion Centre",0.000000,0.075000,0.025000,0.000000,0.00000,0.00000,0.000000,0.025000,0.000000,...,0.150000,0.025000,0.075000,0.025000,0.000000,0.000000,0.000000,0.025000,0.000000,0.000000
9,"First Canadian Place,Underground city",0.000000,0.066667,0.066667,0.000000,0.00000,0.00000,0.022222,0.000000,0.022222,...,0.133333,0.022222,0.066667,0.022222,0.000000,0.044444,0.000000,0.022222,0.000000,0.000000


In [96]:
#Summarize with the top 5 values for every neighborhood
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                venue  freq
0          Restaurant  0.12
1     Thai Restaurant  0.10
2                Café  0.10
3      Breakfast Spot  0.05
4  Seafood Restaurant  0.05


----Berczy Park----
                 venue  freq
0           Restaurant  0.13
1                 Café  0.13
2   Seafood Restaurant  0.13
3  Japanese Restaurant  0.07
4     Greek Restaurant  0.07


----Cabbagetown,St. James Town----
                venue  freq
0          Restaurant  0.16
1                Café  0.11
2         Pizza Place  0.11
3  Italian Restaurant  0.11
4     Thai Restaurant  0.05


----Central Bay Street----
                 venue  freq
0   Italian Restaurant  0.11
1  Japanese Restaurant  0.08
2         Burger Joint  0.08
3       Sandwich Place  0.08
4   Chinese Restaurant  0.06


----Chinatown,Grange Park,Kensington Market----
                           venue  freq
0          Vietnamese Restaurant  0.12
1                           Café  0.12
2  Vegetarian / Vegan Restaura

In [97]:
#Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Show the top 10 venues by neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Restaurant,Thai Restaurant,Café,Pizza Place,Vegetarian / Vegan Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Seafood Restaurant,Breakfast Spot
1,Berczy Park,Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Eastern European Restaurant,Breakfast Spot,Greek Restaurant,French Restaurant,BBQ Joint,Thai Restaurant
2,"Cabbagetown,St. James Town",Restaurant,Italian Restaurant,Café,Pizza Place,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,Gastropub,Thai Restaurant,Taiwanese Restaurant
3,Central Bay Street,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Chinese Restaurant,Thai Restaurant,Seafood Restaurant,Sushi Restaurant
4,"Chinatown,Grange Park,Kensington Market",Vietnamese Restaurant,Café,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Comfort Food Restaurant,Pizza Place,Chinese Restaurant,Burger Joint,Bed & Breakfast


In [98]:
#Now, let's cluster the neighborhoods
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 4, 4, 3, 4, 1, 1, 1])

In [99]:
#Include cluster and top 10 venues
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted
Toronto_merged = Downtown_data
Toronto_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,2.0,Restaurant,Café,Mexican Restaurant,Asian Restaurant,French Restaurant,Breakfast Spot,Wings Joint,Doner Restaurant,Fried Chicken Joint,Filipino Restaurant
1,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,4.0,Middle Eastern Restaurant,Japanese Restaurant,Café,Restaurant,Diner,Italian Restaurant,Pizza Place,Ramen Restaurant,Fast Food Restaurant,Sandwich Place
2,Downtown Toronto,St. James Town,43.651494,-79.375418,1.0,Restaurant,Café,Italian Restaurant,Diner,Breakfast Spot,American Restaurant,Thai Restaurant,Gastropub,BBQ Joint,Seafood Restaurant
3,Downtown Toronto,Berczy Park,43.644771,-79.373306,1.0,Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Eastern European Restaurant,Breakfast Spot,Greek Restaurant,French Restaurant,BBQ Joint,Thai Restaurant
4,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4.0,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Chinese Restaurant,Thai Restaurant,Seafood Restaurant,Sushi Restaurant


In [100]:
#Get rid of the NaN values
Toronto_merged = Toronto_merged[Toronto_merged['Cluster Labels'].notna()]
Toronto_merged = Toronto_merged.astype({"Cluster Labels": int})

In [138]:
#Make the map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

## Let's take a look at the clusters

In [102]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Harbord,University of Toronto",Café,Italian Restaurant,Japanese Restaurant,Restaurant,Chinese Restaurant,Sandwich Place,Comfort Food Restaurant,French Restaurant,Sushi Restaurant,Belgian Restaurant


In [103]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,St. James Town,Restaurant,Café,Italian Restaurant,Diner,Breakfast Spot,American Restaurant,Thai Restaurant,Gastropub,BBQ Joint,Seafood Restaurant
3,Berczy Park,Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Eastern European Restaurant,Breakfast Spot,Greek Restaurant,French Restaurant,BBQ Joint,Thai Restaurant
6,"Adelaide,King,Richmond",Restaurant,Thai Restaurant,Café,Pizza Place,Vegetarian / Vegan Restaurant,American Restaurant,Asian Restaurant,Sushi Restaurant,Seafood Restaurant,Breakfast Spot
7,"Harbourfront East,Toronto Islands,Union Station",Café,Italian Restaurant,Restaurant,Fried Chicken Joint,Pizza Place,Mexican Restaurant,Indian Restaurant,New American Restaurant,Japanese Restaurant,Chinese Restaurant
8,"Design Exchange,Toronto Dominion Centre",Café,Restaurant,Seafood Restaurant,Gastropub,Italian Restaurant,Japanese Restaurant,American Restaurant,Pizza Place,Sushi Restaurant,Chinese Restaurant
9,"Commerce Court,Victoria Hotel",Café,Restaurant,American Restaurant,Seafood Restaurant,Japanese Restaurant,Gastropub,Italian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Breakfast Spot
14,Stn A PO Boxes 25 The Esplanade,Café,Seafood Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,Breakfast Spot,Comfort Food Restaurant,Eastern European Restaurant,Molecular Gastronomy Restaurant,Pizza Place
15,"Cabbagetown,St. James Town",Restaurant,Italian Restaurant,Café,Pizza Place,Caribbean Restaurant,Indian Restaurant,Japanese Restaurant,Gastropub,Thai Restaurant,Taiwanese Restaurant
16,"First Canadian Place,Underground city",Café,Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Gastropub,Asian Restaurant,Pizza Place,Thai Restaurant,Sandwich Place


In [104]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",Restaurant,Café,Mexican Restaurant,Asian Restaurant,French Restaurant,Breakfast Spot,Wings Joint,Doner Restaurant,Fried Chicken Joint,Filipino Restaurant


In [105]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Christie,Café,Diner,Italian Restaurant,Restaurant,Wings Joint,Fried Chicken Joint,French Restaurant,Filipino Restaurant,Fast Food Restaurant,Falafel Restaurant


In [106]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Ryerson,Garden District",Middle Eastern Restaurant,Japanese Restaurant,Café,Restaurant,Diner,Italian Restaurant,Pizza Place,Ramen Restaurant,Fast Food Restaurant,Sandwich Place
4,Central Bay Street,Italian Restaurant,Japanese Restaurant,Sandwich Place,Burger Joint,Café,Middle Eastern Restaurant,Chinese Restaurant,Thai Restaurant,Seafood Restaurant,Sushi Restaurant
11,"Chinatown,Grange Park,Kensington Market",Vietnamese Restaurant,Café,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Comfort Food Restaurant,Pizza Place,Chinese Restaurant,Burger Joint,Bed & Breakfast
17,Church and Wellesley,Japanese Restaurant,Sushi Restaurant,Restaurant,Café,Mediterranean Restaurant,Gastropub,Breakfast Spot,Burger Joint,American Restaurant,Caribbean Restaurant


# Section 2:  Get data about Lexington, KY


In [107]:
#Location of Lexington
LexLat=38.0406
LexLong=-84.5037

#Read in Lexington neighborhood data (pulled from Google)
lex=pd.read_csv('C:/Users/kevmo/Downloads/LexZip.csv',encoding='latin1')
#.set_index("Postcode")
lex

,Postcode,Neighborhood,Latitude,Longitude
0,40502,Chevy Chase,38.0134,-84.485
1,40503,"Lansdowne, Open Gates",38.0103,-84.532
2,40504,"Cardinal Hill, Gardenside, Colony",38.0370,-84.537
3,40505,Eastland,38.0541,-84.454
4,40507,Central Downtown,38.0468,-84.494
5,40509,Kenwood,37.9924,-84.375
6,40511,Meadowthorpe,38.1249,-84.501
7,40513,Southern Park,38.0142,-84.616
8,40517,Stoneybrook,37.9817,-84.485


In [108]:
# create map of Lexington using latitude and longitude values
map_Lexington = folium.Map(location=[LexLat, LexLong], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(lex['Latitude'], lex['Longitude'], lex['Neighborhood']):
 
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Lexington)  
    
map_Lexington

In [109]:
#Get the Foursquare URL for Lexington

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
Lexurl = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    LexLat, 
    LexLong, 
    radius, 
    LIMIT)
Lexurl # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LRO1TXA4U31JLAUVFFCBHTC2XP3A3JIB1CCALQQYTNKBIPMK&client_secret=FU2W2TWVBIRQK1BRFRZSBIQMEJQG0NTMSW1ABLT4XUOQR1RA&v=20180605&ll=38.0406,-84.5037&radius=500&limit=100'

In [110]:
Lexresults = requests.get(Lexurl).json()
Lexresults

{'meta': {'code': 200, 'requestId': '5e780c6eda9e14002215d9fe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'University of Kentucky',
  'headerFullLocation': 'University of Kentucky, Lexington-Fayette',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 48,
  'suggestedBounds': {'ne': {'lat': 38.0451000045, 'lng': -84.49799691585858},
   'sw': {'lat': 38.036099995499995, 'lng': -84.50940308414141}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b46c6ebf964a5201e2826e3',
       'name': "Raising Cane's Chicken Fingers",
       'location': {'address': '544 S Upper St',
        'crossStreet': 'Ave of Champions',
        'lat'

In [111]:
Lexvenues = Lexresults['response']['groups'][0]['items']
    
Lexnearby_venues = json_normalize(Lexvenues) # flatten JSON

# filter columns
Lexfiltered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Lexnearby_venues = Lexnearby_venues.loc[:, filtered_columns]

# filter the category for each row
Lexnearby_venues['venue.categories'] = Lexnearby_venues.apply(get_category_type, axis=1)

# clean columns
Lexnearby_venues.columns = [col.split(".")[-1] for col in Lexnearby_venues.columns]

Lexnearby_venues.head(50)

,name,categories,lat,lng
0,Raising Cane's Chicken Fingers,Fried Chicken Joint,38.042456,-84.504242
1,Han Woo Ri,Korean Restaurant,38.042061,-84.503358
2,Mellow Mushroom,Pizza Place,38.043498,-84.503424
3,Chipotle Mexican Grill,Mexican Restaurant,38.042702,-84.502686
4,The Local Taco,Taco Place,38.043130,-84.501811
5,Sav's Grill & West African Cuisine,African Restaurant,38.043252,-84.501477
6,CD Central,Record Shop,38.042057,-84.503364
7,The Tin Roof,Bar,38.043294,-84.501632
8,Joe Bologna's Pizza,Pizza Place,38.043815,-84.502019
9,Coffea,Coffee Shop,38.038205,-84.499532


In [112]:
#Count the venues
print('{} venues were returned by Foursquare.'.format(Lexnearby_venues.shape[0]))

48 venues were returned by Foursquare.


In [113]:
Lex_venues = getNearbyVenues(names=lex['Neighborhood'],
                                   latitudes=lex['Latitude'],
                                   longitudes=lex['Longitude']
                                  )

Chevy Chase
Lansdowne, Open Gates
Cardinal Hill, Gardenside, Colony
Eastland
Central Downtown
Kenwood
Meadowthorpe
Southern Park
Stoneybrook


In [114]:
#How big is the dataframe?
print(Lex_venues.shape)
Lex_venues.head(50)

(49, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Lansdowne, Open Gates",38.0103,-84.532,Private Residence,38.010335,-84.532704,Music Venue
1,"Lansdowne, Open Gates",38.0103,-84.532,Hill N' Dale Park,38.012861,-84.530293,Playground
2,"Lansdowne, Open Gates",38.0103,-84.532,REEDS Jewelers,38.009291,-84.528442,Jewelry Store
3,"Lansdowne, Open Gates",38.0103,-84.532,Slones Market,38.013442,-84.528379,Grocery Store
4,"Lansdowne, Open Gates",38.0103,-84.532,Twenty Twelve,38.008719,-84.537033,Art Gallery
5,"Cardinal Hill, Gardenside, Colony",38.0370,-84.537,Ky Eye Institute,38.036751,-84.535355,Optical Shop
6,"Cardinal Hill, Gardenside, Colony",38.0370,-84.537,Bluegrass Rooter,38.033665,-84.536706,Home Service
7,Eastland,38.0541,-84.454,Kenawood Park,38.055560,-84.450703,Park
8,Eastland,38.0541,-84.454,Rexel Electrical And Datacom,38.050471,-84.453647,Electronics Store
9,Eastland,38.0541,-84.454,Donkmaron Mexican Restaurant and Seafood,38.053939,-84.459070,Restaurant


In [115]:
#How many unique categories?
print('There are {} unique categories.'.format(len(Lex_venues['Venue Category'].unique())))

There are 40 unique categories.


In [116]:
#Let's limit the venues to restaurants.  Note that some restaurant categories don't include the word 'restaurant'

Lex_venues=Lex_venues[Lex_venues['Venue Category'].str.contains(Restonly)]
Lex_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
9,Eastland,38.0541,-84.454,Donkmaron Mexican Restaurant and Seafood,38.053939,-84.459070,Restaurant
12,Central Downtown,38.0468,-84.494,Corto Y Lima,38.047130,-84.496350,Latin American Restaurant
13,Central Downtown,38.0468,-84.494,Alfalfa Restaurant,38.045583,-84.496060,Vegetarian / Vegan Restaurant
15,Central Downtown,38.0468,-84.494,Le Deauville,38.048167,-84.494562,French Restaurant
17,Central Downtown,38.0468,-84.494,The Lexington Diner,38.047712,-84.497241,Diner
18,Central Downtown,38.0468,-84.494,Doodles,38.049293,-84.492682,Breakfast Spot
20,Central Downtown,38.0468,-84.494,Dad's Favorite,38.044137,-84.495215,Sandwich Place
22,Central Downtown,38.0468,-84.494,Third Street Stuff & Coffee,38.049382,-84.493081,Café
25,Central Downtown,38.0468,-84.494,Lockbox,38.047176,-84.497674,Molecular Gastronomy Restaurant
26,Central Downtown,38.0468,-84.494,Goodfellas Pizzeria,38.048418,-84.498585,Pizza Place


In [117]:
#Let's look at each neighborhood

# one hot encoding
Lex_onehot = pd.get_dummies(Lex_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Lex_onehot['Neighborhood'] = Lex_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Lex_onehot.columns[-1]] + list(Lex_onehot.columns[:-1])
Lex_onehot = Lex_onehot[fixed_columns]

Lex_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Breakfast Spot,Café,Diner,French Restaurant,Gastropub,Hot Dog Joint,Latin American Restaurant,Molecular Gastronomy Restaurant,Pizza Place,Restaurant,Sandwich Place,Vegetarian / Vegan Restaurant
9,Eastland,0,0,0,0,0,0,0,0,0,0,0,1,0,0
12,Central Downtown,0,0,0,0,0,0,0,0,1,0,0,0,0,0
13,Central Downtown,0,0,0,0,0,0,0,0,0,0,0,0,0,1
15,Central Downtown,0,0,0,0,0,1,0,0,0,0,0,0,0,0
17,Central Downtown,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [118]:
#How big is the dataframe?
Lex_onehot.shape

(15, 15)

In [119]:
#Let's limit our scope to restaurants
#Lex_restaurants=Lex_onehot.filter(regex='Restaurant')
#Lex_restaurants=Lex_onehot.filter(regex='Restaurant|Caf|Joint|Gastropub|Diner|Pizza|Breakfast|Sandwich')
#Lex_restaurants

In [120]:
#Lex_restaurants.shape


#### Conclusion 1:  Remember that Toronto had 479 restaurants.  Toronto is 9.1 times the size of Lexington, so we would expect Lexington to have around 52 restaurants.  49 is pretty close, so Lexington and Toronto have similar food scenes in terms of number of restaurants.

In [121]:
#Group everything by neighborhood and take the average of each category
Lex_grouped = Lex_onehot.groupby('Neighborhood').mean().reset_index()
Lex_grouped_rest=Lex_grouped.filter(regex='Restaurant')
Lex_grouped_rest

,American Restaurant,Asian Restaurant,French Restaurant,Latin American Restaurant,Molecular Gastronomy Restaurant,Restaurant,Vegetarian / Vegan Restaurant
0,0.076923,0.076923,0.076923,0.076923,0.076923,0.0,0.076923
1,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [122]:
#Group everything by neighborhood and take the total of each category
Lex_groupedt = Lex_onehot.groupby('Neighborhood').sum().reset_index()
Lex_grouped_restt=Lex_groupedt.filter(regex='Restaurant')
Lex_grouped_restt.shape


(3, 7)

In [135]:
#Let's look at the different types of restaurants in Lexington
for col in Lex_onehot.columns:
    print(col)


Neighborhood
American Restaurant
Asian Restaurant
Breakfast Spot
Café
Diner
French Restaurant
Gastropub
Hot Dog Joint
Latin American Restaurant
Molecular Gastronomy Restaurant
Pizza Place
Restaurant
Sandwich Place
Vegetarian / Vegan Restaurant


In [124]:
#Let's look at the different types of restaurants in Toronto
for col in Toronto_restaurants.columns:
    print(col)

Afghan Restaurant
American Restaurant
Asian Restaurant
BBQ Joint
Bed & Breakfast
Belgian Restaurant
Brazilian Restaurant
Breakfast Spot
Burger Joint
Café
Caribbean Restaurant
Chinese Restaurant
Colombian Restaurant
Comfort Food Restaurant
Diner
Doner Restaurant
Dumpling Restaurant
Eastern European Restaurant
Empanada Restaurant
Ethiopian Restaurant
Falafel Restaurant
Fast Food Restaurant
Filipino Restaurant
French Restaurant
Fried Chicken Joint
Gaming Cafe
Gastropub
German Restaurant
Gluten-free Restaurant
Greek Restaurant
Hotpot Restaurant
Indian Restaurant
Italian Restaurant
Japanese Restaurant
Korean Restaurant
Latin American Restaurant
Mediterranean Restaurant
Mexican Restaurant
Middle Eastern Restaurant
Modern European Restaurant
Molecular Gastronomy Restaurant
New American Restaurant
Pizza Place
Polish Restaurant
Portuguese Restaurant
Ramen Restaurant
Restaurant
Sandwich Place
Seafood Restaurant
Sushi Restaurant
Taiwanese Restaurant
Thai Restaurant
Theme Restaurant
Vegetarian / V

In [137]:
Toronto_restaurants.shape

(479, 56)

### Conclusion 2:  Toronto has much more variety in restaurants than Lexington

### Let's look at the same clustering algorithm applied to Lexington

In [125]:
#Summarize with the top 5 values for every neighborhood
num_top_venues = 5

for hood in Lex_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Lex_grouped[Lex_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Downtown----
                 venue  freq
0  American Restaurant  0.08
1     Asian Restaurant  0.08
2       Breakfast Spot  0.08
3                 Café  0.08
4                Diner  0.08


----Eastland----
                 venue  freq
0           Restaurant   1.0
1  American Restaurant   0.0
2     Asian Restaurant   0.0
3       Breakfast Spot   0.0
4                 Café   0.0


----Southern Park----
                 venue  freq
0                Diner   1.0
1  American Restaurant   0.0
2     Asian Restaurant   0.0
3       Breakfast Spot   0.0
4                 Café   0.0




In [126]:
#Show the top 10 venues by neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Lex_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Lex_neighborhoods_venues_sorted['Neighborhood'] = Lex_grouped['Neighborhood']

for ind in np.arange(Lex_grouped.shape[0]):
    Lex_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Lex_grouped.iloc[ind, :], num_top_venues)

Lex_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Downtown,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner,Café
1,Eastland,Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner
2,Southern Park,Diner,Vegetarian / Vegan Restaurant,Sandwich Place,Restaurant,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant


In [127]:
#Now, let's cluster the neighborhoods
# set number of clusters
Lkclusters = 3

Lex_grouped_clustering = Lex_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=Lkclusters, random_state=0).fit(Lex_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1])

In [128]:
#Include cluster and top 10 venues
# add clustering labels
Lex_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Lex_neighborhoods_venues_sorted
Lex_merged = lex
Lex_merged

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Lex_merged = Lex_merged.join(Lex_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Lex_merged.head() 

,Postcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,40502,Chevy Chase,38.0134,-84.485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40503,"Lansdowne, Open Gates",38.0103,-84.532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40504,"Cardinal Hill, Gardenside, Colony",38.0370,-84.537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40505,Eastland,38.0541,-84.454,2.0,Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner
4,40507,Central Downtown,38.0468,-84.494,0.0,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner,Café


In [129]:
#Get rid of the NaN values and make sure that the remaining values are integers
Lex_merged = Lex_merged[Lex_merged['Cluster Labels'].notna()]
Lex_merged = Lex_merged.astype({"Cluster Labels": int})

In [130]:
#Make the map
# create map
map_clusters = folium.Map(location=[LexLat, LexLong], zoom_start=11)

# set color scheme for the clusters
x = np.arange(Lkclusters)
ys = [i + x + (i*x)**2 for i in range(Lkclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Lex_merged['Latitude'], Lex_merged['Longitude'], Lex_merged['Neighborhood'], Lex_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

## Let's take a closer look at the clusters

In [131]:
Lex_merged.loc[Lex_merged['Cluster Labels'] == 0, Lex_merged.columns[[1] + list(range(5, Lex_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Downtown,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner,Café


In [132]:
Lex_merged.loc[Lex_merged['Cluster Labels'] == 1, Lex_merged.columns[[1] + list(range(5, Lex_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Southern Park,Diner,Vegetarian / Vegan Restaurant,Sandwich Place,Restaurant,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant


In [133]:
Lex_merged.loc[Lex_merged['Cluster Labels'] == 2, Lex_merged.columns[[1] + list(range(5, Lex_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Eastland,Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Pizza Place,Molecular Gastronomy Restaurant,Latin American Restaurant,Hot Dog Joint,Gastropub,French Restaurant,Diner


In [134]:
Lex_merged.loc[Lex_merged['Cluster Labels'] == 3, Lex_merged.columns[[1] + list(range(5, Lex_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
